In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/a.boldovskaya/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
data = pd.read_csv('./data_seta/kontur_train.csv')

In [6]:
data.head()

,id,name,tare
0,0,Котлеты МЛМ из говядины 335г,коробка
1,1,Победа Вкуса конфеты Мишки в лесу 250г(КФ ПОБЕ...,коробка
2,2,"ТВОРОГ (ЮНИМИЛК) ""ПРОСТОКВАШИНО"" ЗЕРНЕНЫЙ 130Г...",стаканчик
3,3,Сыр Плавленый Веселый Молочник с Грибами 190г ...,контейнер
4,4,Жевательный мармелад Маша и медведь буквы 100г,пакет без формы


In [7]:
tares = data.tare.unique()
key_value = list(map(lambda t: (t[1], t[0]), enumerate(tares)))
tares_to_integer = dict(key_value)
integers = list(map(lambda t: tares_to_integer[t], data.tare))
data['tare_value'] = integers

In [8]:
data.name = data.name.apply(lambda n: n.lower())

tokenizer = nltk.RegexpTokenizer(r'\w+')
data['tokens'] = data.name.apply(lambda n: tokenizer.tokenize(n))
data.head()

,id,name,tare,tare_value,tokens
0,0,котлеты млм из говядины 335г,коробка,0,"[котлеты, млм, из, говядины, 335г]"
1,1,победа вкуса конфеты мишки в лесу 250г(кф побе...,коробка,0,"[победа, вкуса, конфеты, мишки, в, лесу, 250г,..."
2,2,"творог (юнимилк) ""простоквашино"" зерненый 130г...",стаканчик,1,"[творог, юнимилк, простоквашино, зерненый, 130..."
3,3,сыр плавленый веселый молочник с грибами 190г ...,контейнер,2,"[сыр, плавленый, веселый, молочник, с, грибами..."
4,4,жевательный мармелад маша и медведь буквы 100г,пакет без формы,3,"[жевательный, мармелад, маша, и, медведь, букв..."


In [9]:
def isNotJunkWord(word):
    if any(char.isdigit() for char in word):
        return False
    if len(word) < 2:
        return False
    return True

def where(arr, condition):
    new = []
    for element in arr:
        if condition(element) == True:
            new.append(element)
    return new

In [10]:
def get_words_by_frequency(data):
    words = np.array([item for sublist in data.tokens for item in sublist])
    words = where(words, isNotJunkWord)

    unique_words, counts = np.unique(words, return_counts=True)

    return list(sorted(zip(unique_words, counts), key=lambda pair: pair[1], reverse=True))

In [11]:
def calc_coef(tokens, w):
    coef = 0
    for pair in w:
        if pair[0] in tokens:
            coef += int(pair[1]) / len(w)
    return coef

def calc_coefs(data, tare_value):
    words_by_frequency = get_words_by_frequency(data[data['tare_value'] == tare_value])
    print(words_by_frequency)
    threshold = len(words_by_frequency) / 200
    words_by_frequency = list(filter(lambda pair: pair[1] > threshold, words_by_frequency))
    return data.tokens.apply(lambda tok: calc_coef(tok, words_by_frequency))

In [20]:
%%time

for i in range(0, 21):
    key = str(i)+'_coef'
    data[key] = calc_coefs(data, i)

AttributeError: 'DataFrame' object has no attribute 'tokens'

In [15]:
data = data.drop(['tare', 'name', 'tokens'], axis=1)
data.head()

KeyError: "['tare' 'name' 'tokens'] not found in axis"

In [18]:
data.iloc[:, 2:]

,0_coef,1_coef,2_coef,3_coef,4_coef,5_coef,6_coef,7_coef,8_coef,9_coef,...,11_coef,12_coef,13_coef,14_coef,15_coef,16_coef,17_coef,18_coef,19_coef,20_coef
0,0.550000,0.183280,0.130719,0.500000,0.203966,0.695035,0.163265,0.082090,0.000000,0.117647,...,0.078652,0.297101,0.000000,0.000000,0.051613,0.000000,0.484321,0.058442,0.129187,0.000000
1,0.734615,0.000000,0.000000,1.394180,0.000000,0.000000,0.000000,0.104478,0.000000,0.670588,...,0.101124,0.047101,0.077982,0.558824,0.148387,0.000000,0.062718,0.000000,0.000000,0.000000
2,0.088462,1.266881,0.379085,0.171958,0.000000,0.092199,0.000000,0.000000,0.000000,0.643137,...,0.000000,0.000000,0.211009,0.000000,0.000000,0.000000,0.000000,0.097403,0.000000,0.000000
3,1.038462,0.286174,2.588235,0.248677,0.000000,0.546099,0.500000,0.082090,0.000000,0.286275,...,0.595506,0.000000,0.022936,0.029412,1.083871,0.000000,0.041812,0.019481,0.066986,0.000000
4,0.200000,0.131833,0.045752,0.145503,0.000000,0.000000,0.000000,0.000000,0.000000,0.494118,...,0.056180,0.000000,0.091743,0.000000,0.000000,0.018634,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40643,0.400000,0.147910,0.320261,0.537037,5.985836,0.070922,0.000000,0.000000,0.000000,0.372549,...,0.224719,0.833333,0.238532,0.330882,0.045161,0.018634,0.146341,0.019481,0.000000,0.098361
40644,0.630769,0.263666,0.196078,1.894180,1.362606,0.234043,0.295918,0.059701,0.119497,1.015686,...,0.258427,0.282609,0.224771,0.058824,0.103226,0.130435,0.372822,0.077922,0.076555,0.172131
40645,0.165385,0.144695,0.000000,0.761905,1.677054,0.035461,0.000000,0.000000,0.025157,0.172549,...,0.000000,0.152174,0.000000,0.036765,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000
40646,0.000000,0.000000,0.111111,0.000000,5.830028,0.000000,0.061224,0.000000,0.000000,0.109804,...,0.000000,0.166667,0.022936,0.036765,0.000000,0.000000,0.000000,0.000000,0.000000,0.188525


In [ ]:
from sklearn import cross_validation, datasets, grid_search, linear_model, metrics
from sklearn.ensemble import RandomForestClassifier

In [ ]:

train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(data.iloc[:, 2:], data.iloc[:, 1:2],
                                                                                    random_state=0, test_size=0.3)

classifier = RandomForestClassifier(random_state = 0)
parameters_grid = {
    'class_weight': [None],
    'bootstrap': [True],
    'criterion': ['gini'],
    'max_depth': [35]
}
cv = cross_validation.StratifiedShuffleSplit(train_labels, n_iter = 5, test_size = 0.2, random_state = 0)

In [ ]:
grid_cv = grid_search.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_iter = 1,
                                                   random_state = 0)

In [ ]:
%%time
grid_cv.fit(train_data, train_labels)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

In [ ]:
grid_cv.grid_scores_

In [ ]:
print(metrics.classification_report(test_labels, grid_cv.predict(test_data)))

In [ ]:
matrix = metrics.confusion_matrix(test_labels, grid_cv.predict(test_data))

df_cm = pd.DataFrame(matrix)
plt.figure(figsize = (20,14))
sns.heatmap(df_cm, annot=True, vmax=70)

In [ ]:
for_zero = get_words_by_frequency(data[data['tare_value'] == 9])
for_three = get_words_by_frequency(data[data['tare_value'] == 3])

In [ ]:
predicted = grid_cv.predict(test_data)
labels = np.array(test_labels['tare_value'])
indicies = []
for i in range(0, len(predicted)):
    if predicted[i] != labels[i]:
        indicies.append(i)

In [ ]:
new_data = pd.read_csv('train.csv')
new_data = new_data.iloc[test_data.index]


In [ ]:

key_value = list(map(lambda t: (t[1], t[0]), enumerate(tares)))
tares_to_integer = dict(key_value)
integers = list(map(lambda t: tares_to_integer[t], new_data.tare))
new_data['tare_value'] = integers
new_data['predicted'] = predicted
new_data.head()

In [ ]:
for_zero = get_words_by_frequency(data[data['tare_value'] == 6])
for_three = get_words_by_frequency(data[data['tare_value'] == 17])

for_zero[:10]


In [ ]:
for_three[:10]
